In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import json

from oraclai.llm_guide.utils import *
from oraclai.llm_guide.init_utils import *
from oraclai.llm_guide.infer_utils import *
from oraclai.llm_guide.loop_utils import *
from oraclai.llm_guide.mongo_utils import *
from oraclai.llm_guide.manual_ndd import *

In [3]:
load_dotenv()

True

In [ ]:
APP_NAME = 

In [4]:
crawl_context: CrawlContext = CrawlContext()
crawl_context = crawl_context.set_temp_var('config_path', f'./configs/pet-clinic.json')

config: dict = read_config(config_path=crawl_context.temp_vars.get('config_path', None))
config_obj: Config = Config.from_dict(config)

if config_obj.base_url is None:
    raise ValueError('base_url is required in config')

crawl_context = crawl_context.set_config(config_obj)

driver = initialize_driver(config_obj)
crawl_context = crawl_context.set_driver(driver)

crawl_context = initialize_variables(crawl_context)

  Reading config from ./configs/pet-clinic.json
  Initializing driver
  Initializing classes with initial state


In [5]:
def force_critical(state: State, action: Action) -> bool:
    pair = (state.url, action.get_id())
    return pair in FORBIDDEN_ACTIONS

In [6]:
LOOP_COUNTER = 0
LOOP_LIMIT = 100
FIRST_RUN = True
VISIT_COUNTER = {}

In [7]:
# for shared actions such as header or footer
EXECUTE_ONCE = {
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/DIV[1]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[1]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[1]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[2]/UL[1]/LI[2]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[3]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[3]/UL[1]/LI[1]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[3]/UL[1]/LI[2]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[4]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[1]/NAV[1]/DIV[1]/UL[1]/LI[5]/A[1]': 0,
    '//BODY/APP-ROOT[1]/DIV[2]/APP-VISIT-ADD[1]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[1]/DIV[1]/MAT-DATEPICKER-TOGGLE[1]/BUTTON[1]': 1,
    '//BODY/APP-ROOT[1]/DIV[2]/APP-VISIT-EDIT[1]/DIV[1]/DIV[1]/FORM[1]/DIV[1]/DIV[1]/DIV[1]/MAT-DATEPICKER-TOGGLE[1]/BUTTON[1]': 1,
    '//BODY/APP-ROOT[1]/DIV[2]/APP-PET-EDIT[1]/DIV[1]/DIV[1]/FORM[1]/DIV[4]/DIV[1]/MAT-DATEPICKER-TOGGLE[1]/BUTTON[1]': 1
}

In [8]:
while len(crawl_context.crawl_queue) > 0:
    state: State = crawl_context.crawl_queue.dequeue()
    logger.info(f"Visiting state {state.get_id(StateIdEvaluator.BY_ACTIONS)}")
    crawl_context.state_machine.set_current_state(state)
    
    current_state: State = crawl_context.state_machine.get_current_state()
    current_actions: list[Action] = current_state.get_actions()

    crawl_context.load_state(current_state)

    logger.info('Extracting state context using LLM')

    state_context = extract_state_context(
        crawl_context,
        current_state,
        current_state.crawl_path.get_state(-1) if len(current_state.crawl_path) > 0 else None,
        current_state.crawl_path.get_action(-1) if len(current_state.crawl_path) > 0 else None,
    )
    current_state.set_context(state_context)

    for action in current_actions:
        LOOP_COUNTER += 1
        
        logger.info(f'Executing action {action.element.outerHTML}')

        is_critical = is_action_critical(action) or force_critical(current_state, action)

        if action.get_id() in EXECUTE_ONCE and EXECUTE_ONCE[action.get_id()] == 0:
            EXECUTE_ONCE[action.get_id()] = 1
        elif action.get_id() in EXECUTE_ONCE:
            is_critical = True
        
        if not is_critical:
            if action.get_type().get_value() == 'form':
                values = create_form_filling_values(action)
                action.set_params(values)
            
            action.execute(crawl_context.driver)

            new_actions = []

            for i in range(10):
                try:
                    new_actions: list[Action] = CandidateActionExtractor.extract_candidate_actions(crawl_context.driver)
                    break
                except:
                    time.sleep(0.1)
            if len(new_actions) == 0:
                raise Exception("no new actions")
            
            new_state: State = crawl_context.create_state_from_driver(new_actions)

            VISIT_COUNTER, visit_forbidden_bool = is_visit_forbidden(new_state, VISIT_COUNTER)
            
            if not visit_forbidden_bool and not is_state_in_graph(crawl_context, new_state):
                print('Adding state', new_state.get_id(StateIdEvaluator.BY_ACTIONS))
                crawl_context.crawl_queue.enqueue(new_state)
                crawl_context.state_machine.add_state_from_current_state(new_state, action)
        
        crawl_context.load_state(crawl_context.state_machine.get_current_state())

        logger.info("")

crawl_context.driver.quit()

  Visiting state 00a45313f06e84ca1980f9431a56fe5f586ba9cc4842c602afb00191ee4aefbe
  Extracting state context using LLM
  Saved screenshot to ./tmp/screenshot_00a45313f06e84ca1980f9431a56fe5f586ba9cc4842c602afb00191ee4aefbe.png
  Prompt:
  The description of the website is: None
The previous state was: None. This is the first state.
The previous action was: None. This is the first state.
  Response:
  A homepage for a veterinary clinic or pet care service, featuring navigation options and a welcoming image of animals.
  
  Executing action <a _ngcontent-ng-c2241170354="" href="#" class="navbar-brand"><span _ngcontent-ng-c2241170354=""></span></a>
  Prompt:
  <a _ngcontent-ng-c2241170354="" href="#" class="navbar-brand"><span _ngcontent-ng-c2241170354=""></span></a>
  Response:
  False
  
Adding state 00a45313f06e84ca1980f9431a56fe5f586ba9cc4842c602afb00191ee4aefbe
  
  Executing action <a _ngcontent-ng-c2241170354="" routerlink="welcome" title="home page" data-testid="c1-navbar-home" ng

In [9]:
print(LOOP_COUNTER)

232


In [10]:
for state_id, state_obj in crawl_context.state_machine.state_graph.states.items():
    print(state_id, state_obj.url)

00a45313f06e84ca1980f9431a56fe5f586ba9cc4842c602afb00191ee4aefbe http://localhost:4200/petclinic/
c49f72f2405b8bf804d5d9d066b673cfdcd49924f84b36ffb4fc0f995e8ac9a2 http://localhost:4200/petclinic/
cdbea3b4ec13b0eb69faa3f0742c34b09ef593765203ab7a6ef160dc31560181 http://localhost:4200/petclinic/
8fada37135ebc534e7af8028a566e46006f60f98946363e456e878d010d588b2 http://localhost:4200/petclinic/pettypes
99dc53da76e510f3161d69b81ed52d9cb33620873d353f1903ec3a5694ac9a42 http://localhost:4200/petclinic/specialties
f8d135e58191af1833d61355bd8096e04354f73d1150b2d787caf25a738988a6 http://localhost:4200/petclinic/owners
c103741155444628d7a552c808f3e1ad8ec77ae360575905bbda024b050c5dc7 http://localhost:4200/petclinic/owners/add
249cb08c976b5574aedb9ec5997badfc31a03f3231df960fc2d4a1bdeae0740f http://localhost:4200/petclinic/vets
edc4fcdc29e01f09206ae2b69b161c0d8b46862679390cf44324c653a1662538 http://localhost:4200/petclinic/vets/add
e061ff871c241ba7a1781d780eed0e1f0a47f9f0a98a4e708ef7d579c6ee8336 http:/

In [11]:
states_converted = {}

for state_id, state_obj in crawl_context.state_machine.state_graph.states.items():
    states_converted[state_id] = {
        'url': state_obj.url,
        'context': state_obj.context,
        'actions': [{
                'type': a.action_type.get_value(),
                'id': a.element.get_id(),
                'outerHTML': clean_children_html(a.element.outerHTML),
                'testId': a.element.test_id
            } for a in state_obj.get_actions()
        ],
        'prev_state': state_obj.crawl_path.get_state(-1).get_id() if len(state.crawl_path) > 0 else None,
        'prev_action': state_obj.crawl_path.get_action(-1).get_id() if len(state.crawl_path) > 0 else None,
    }


adj_list_converted = {}

for state_id, neighbor_list in crawl_context.state_machine.state_graph.adjacency_list.items():
    adj_list_converted[state_id] = {}

    for action_obj, n_state_id in neighbor_list.items():
        adj_list_converted[state_id][action_obj.get_id()] = n_state_id

In [12]:
json.dump(
    {
        'nodes': states_converted,
        'edges': adj_list_converted
    },
    open('./report/pet-clinic.json', 'w+')
)